In [156]:
import pandas as pd
import time

In [157]:
old=pd.read_json('data/old.json')#.reset_index(drop=True)
old.number_of_citations.sum()

1636

Download citations of each paper

In [ ]:
UPDATE=False
if UPDATE:
    for r in old['recid']:
        p=pd.read_json('http://inspirehep.net/search?ln=en&p=refersto:recid:{}&of=recjson'.format(r)
                      ).to_json('data/{}.json'.format(r))
        print(r)
        time.sleep(1)

Download INSPIRE HEP Profile

In [133]:
# TODO: Check if the author has more that 250 records!
exact_author='D.Restrepo.1'
url='http://inspirehep.net/search?p=ea+{}&rg=250&of=recjson'.format(exact_author)
df=pd.read_json(url)
new=df.copy()
df.number_of_citations.sum()

author='restrepo,+d'
df=pd.read_json(
    "http://inspirehep.net/search?ln=en&ln=en&p=a+{}+AND+collection%3Aciteable&rg=250&of=recjson".format(
        author))
new=df.copy()

In [142]:
import sys
ANALYSIS=False
if new['number_of_citations'].sum()>old['number_of_citations'].sum():
    ANALYSIS=True
if not ANALYSIS:
    sys.exit('NO NEW CITES TO ANALYSE!')

In [143]:
old['iid']=old.FIXME_OAI.apply(lambda d: d['id']).str.split(':').str[-1]
new['iid']=old.FIXME_OAI.apply(lambda d: d['id']).str.split(':').str[-1]

In [144]:
old=old.rename({'number_of_citations':'nc'},axis='columns') ##add other relevant columns for analysis

In [145]:
ANALYSIS

True

In [146]:
new=new.merge(old[['nc','iid']],on='iid',how='left')#[['number_of_citations','nc']]

In [147]:
news=new[new['number_of_citations']>new['nc']]

In [ ]:
print('New citations in:')
news['recid']
#['primary_report_number'].apply(lambda l:  [x for x in l if x.find('arXiv')>-1])

In [148]:
for r in news['recid']:
    #Analysis of each citation
    print('===== NEW CITATION FOR {}====='.format(r))
    oldrefersto=pd.read_json('data/{}.json'.format(r))
    refersto=pd.read_json('http://inspirehep.net/search?ln=en&p=refersto:recid:{}&of=recjson'.format(r))
    print( news[news['recid']==r]['title'].values[0] )
    print('with')
    print(news[news['recid']==r]['authors'].apply(lambda l: [d['full_name'] for d in l]).values[0])
    print('WAS CITED BY:')
    newrecid=pd.np.setdiff1d(refersto['recid'],oldrefersto['recid'])
    #newrecid=pd.np.setdiff1d(refersto['recid'],oldrefersto['recid'][:-2])
    for rr in newrecid:
        print('*'*60)
        print( refersto[refersto['recid']==rr]['title'].values[0])
        print( refersto[refersto['recid']==rr]['authors'].apply(lambda l: 
                                            [d['full_name'] for d in l]).values[0])
        print( refersto[refersto['recid']==rr]['primary_report_number'].apply(lambda l: 
                                            [x for x in l if x.find('arXiv')>-1]).values[0] )    
    print('UPDATE CITATIONS (Y)')
    YN=input('Y/N')
    if YN!='N' or YN!='n' or YN!='not' or YN!='Not':
        print('UPDATINF JSON DATA CITATION for {}'.format(r))
        refersto.to_json('data/{}.json'.format(r))

===== NEW CITATION FOR 1615477=====
{'title': 'Expanding the Reach of Heavy Neutrino Searches at the LHC'}
with
['Flórez, Andrés', 'Gui, Kaiwen', 'Gurrola, Alfredo', 'Patiño, Carlos', 'Restrepo, Diego']
WAS CITED BY:
UPDATE CITATIONS (Y)
Y/NN
===== NEW CITATION FOR 1249404=====
{'title': 'Models with radiative neutrino masses and viable dark matter candidates'}
with
['Restrepo, Diego', 'Zapata, Oscar', 'Yaguna, Carlos E.']
WAS CITED BY:
UPDATE CITATIONS (Y)
Y/NN
===== NEW CITATION FOR 1220579=====
{'title': 'Scalar dark matter and fermion coannihilations in the radiative seesaw model'}
with
['Klasen, Michael', 'Yaguna, Carlos E.', 'Ruiz-Alvarez, Jose D.', 'Restrepo, Diego', 'Zapata, Oscar']
WAS CITED BY:
UPDATE CITATIONS (Y)


KeyboardInterrupt: 

In [155]:
print('UPDATE FULL DATABASE for profiles (N)')
YN=input('Y/N')
if YN=='Y' or YN=='y' or YN=='yes' or YN=='Yes':
    df.to_json('data/old.json')

UPDATE FULL DATABASE for profiles (N)
Y/NY
